In [148]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [149]:
import pickle as p
import pandas as pd
a = pd.read_csv("/content/drive/My Drive/NASA/AirQualitySystem.csv")

with open("/content/drive/My Drive/NASA/aod",'rb') as f:
  aod= p.load(f)

with open("/content/drive/My Drive/NASA/merra",'rb') as f:
  merra = p.load(f)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
import folium
import numpy as np

def query(lat, lon, date, time, ground, merra, aod):

  
    """
    query data for given coordinates and time in the ground, meteo, aod datasets

    returns Dataframe that match query, flag for dataset matched

    """
    dt = 0
    off = 0.1
    # ground station data
    a['Latitude']= a['Latitude'].round(2)
    a['Longitude']= a['Longitude'].round(2)

    frame = a.loc[
        (a['Latitude'].between(lat-off,lat+off)) &
        (a['Longitude'].between(lon-off,lon+off)) &
        (a['Longitude']== lon) &
        (a['Date GMT']== date) &
        (a['Time GMT']== time)]

    if len(frame) == 0:
      off = 0.1
      # meteo station data
      merra['Latitude']= merra['Lat'].round(2)
      merra['Longitude']= merra['Lon'].round(2)
      date2 =  date.split('-')
      date2= ''.join(date2)
      
      frame = merra.loc[
          (merra['Latitude'].between(lat-off,lat+off)) &
          (merra['Longitude'].between(lon-off,lon+off)) &
          (merra['UTC_DATE']== int(date2))]
      dt += 1
    if len(frame) == 0:
      off = 1.0
      # satellite data
      aod['Latitude']= aod['Latitude'].round(2)
      aod['Longitude']= aod['Longitude'].round(2)
      y, m, d = date.split('-')
      if m[0]== '0':
        m = m[1]


      frame = aod.loc[       
          (aod['Latitude'].between(lat-off,lat+off)) &
          (aod['Longitude'].between(lon-off,lon+off)) &
          (aod['YYYY']== int(y)) &
          (aod['MM']== int(m)) &
          (aod['DD']== int(d))]
      dt += 1
      
    
    return frame, dt
    


def formatter(row,dt):
  """
  format text for marker in html
  use dt flag to manage different dataframes
  return string
  """
  if dt == 0:  
    cols = ['Latitude', 'Longitude', 
      'Parameter Name', 'Sample Measurement']
  elif dt == 1:
    cols = ['Latitude', 'Longitude','AQI']
  elif dt == 2:
    cols = ['Latitude', 'Longitude','AOD3']
  t = []
  t.append('<ul style="list-style-type:disc;">')
  for c in cols:
    #print(c)
    t.append('<li>')
    t.append('<b>')
    t.append(str(c + ':'))
    t.append('</b>')
    #print(row[c])
    #print(type(frame[c][row]))
    if type(row[c]) == np.float64:
      t.append('%.2f' %row[c])
    else:
      t.append(str(row[c]))
    t.append('</li>')
  t.append('</ul>')
  f = (" ".join(t))
  return f
    
def Marker(row,dt):
  """
  take row of dataframe, dataframe type
  generate new Marker and returns it
  
  """

  text = formatter(row,dt)
  coords = [ row['Latitude'], row['Longitude'] ]
  return folium.Marker(coords, popup=text)
  

def Gen(frame, dt, folium_map):
  """
  takes dataframe, flag of dataframe type and map to update
  return map updated  with new group of markers
  
  """

  group = folium.FeatureGroup("Markers")

  # aod dataframe has coords swapped
  if dt == 2:
    frame.rename(columns={'Latitude':'lat', 'Longitude': 'lon'}, inplace=True)
    frame.rename(columns={'lat': 'Longitude', 'lon': 'Latitude'}, inplace=True)
  for i in range(1,len(frame)-1):
    row = frame.iloc[i][:]
    #print(row)
    marker = Marker(row,dt)
    group.add_child(marker)

  return folium_map.add_child(group)


In [151]:
folium_map = folium.Map(width = '99%',height=800,location=[48, -102],
                        zoom_start=3,
                        tiles="Stamen Terrain",min_lat=7, max_lat=35, min_lon=73, max_lon=90)


ground = query(34.14, -117.85, '2008-01-01', '08:00', a, merra, aod)


Gen(ground[0],ground[1],folium_map)

In [152]:
folium_map2 = folium.Map(width = '99%',height=800,location=[48, -102],
                        zoom_start=3,
                        tiles="Stamen Terrain",min_lat=7, max_lat=35, min_lon=73, max_lon=90)


meteo = query(9.05, 38.76, '2018-12-31', '08:00', a, merra, aod)
Gen(meteo[0],meteo[1],folium_map2)

In [153]:
folium_map3 = folium.Map(width = '99%',height=800,location=[48, -102],
                        zoom_start=3,
                        tiles="Stamen Terrain",min_lat=7, max_lat=35, min_lon=73, max_lon=90)


sat = query(-118.0, 34.0, '2000-3-12', '08:00', a, merra, aod)
Gen(sat[0],sat[1],folium_map3)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
